In [5]:
import pandas as pd
import math
import numpy as np
from datetime import date, time, datetime

In [6]:
class Air():
    def __init__(self, temp ,pressure,  rel_hum = None ,abs_hum = None ): #temparature in celsius, humidity in %, pressure in Pascals
        self.t = temp
        self.PRES = pressure
        self.PWS = self.Count_PWS()    

        if rel_hum:
            self.H = rel_hum/100
            self.abs_hum = self.Count_abs_hum()
        elif abs_hum:
            self.abs_hum = abs_hum
            self.H = self.Count_relative_hum()
        self.Enth = self.Count_enthalpy()
        self.density =1/self.Count_volume()
        self.PP = self.Count_partial_pressure()
        self.dewpt = self.Count_dewpoint()
        
    def __repr__(self):
        print('Temperature = ',self.t, 'Celsius degree') 
        print('Relative humidity = ',self.H*100, '%')
        print('Pressure = ',self.PRES, ' Pa')       
        print('PWS =',self.PWS, ' Pa') 
        print('Absolute humidity = ',self.abs_hum, ' KG water/kg Air') 
        print('Enthalpy = ',self.Enth, ' kJ/kg Air') 
        print('Density = ',self.density ,' kg/m3 Air') 
        print('Partial pressure =',self.PP ,' Pa') 
        print('Dew point = ',self.dewpt, ' celsius degree') 
        return 'End of report'
       
        
    def Count_PWS(self):
        if self.t < -100 or self.t > 200:
            print('Temperature out of limits')
            return None

        T = self.t+273.15
        par_minus = [-5674.535900000001, 6.3925247, -0.009677843, 6.2215701e-07, 2.0747825e-09, -9.484024e-13, 4.1635019] #parameters of air below 0 celsius up to -100
        par0200 = [-5800.220600000001, 1.3914993, -0.048640239, 4.1764768e-05, -1.4452093e-08, 6.5459673] #parameters for air above 0 celsius up to 200

        if self.t<0:
            PWS_down = math.exp((par_minus[0]/T) +par_minus[1] +par_minus[2]*T +par_minus[3]*T**2 +par_minus[4]*T**3+par_minus[5]*T**4 +par_minus[6]*np.log(T))
            self.PWS = PWS_down
            return PWS_down
        else:
            PWS_up = math.exp((par0200[0]/T) +par0200[1] +par0200[2]*T +par0200[3]*T**2 +par0200[4]*T**3 +par0200[5]*np.log(T))
            self.PWS = PWS_up
            return PWS_up
    
    def Count_abs_hum(self):
        abs_hum = (0.62198*self.H*self.PWS)/(self.PRES-self.H*self.PWS)
        return abs_hum

    def Count_relative_hum(self):
        H = (self.abs_hum*self.PRES)/(self.abs_hum*self.PWS+0.62198*self.PWS)
        return H
    
    def Count_enthalpy(self):
        Enthalpy =  1.006*self.t+self.abs_hum *(2500.77+self.t*1.82)
        return Enthalpy
    
    def Count_volume(self):
        T=self.t+273.15
        volume = 0.2871*T*(1+1.6078*self.abs_hum)/(self.PRES/1000)
        return volume
    
    def Count_partial_pressure(self):
        PP = self.PRES*self.abs_hum/(0.62198+self.abs_hum)
        return PP

    def Count_dewpoint(self):
        #dew point coeficients
        coef = [6.54, 14.526, 0.7389, 0.09486, 0.4569]

        if self.t<0:
            Dewpt = 6.09+12.608*math.log(self.PP/1000)+0.4959*math.log(self.PP/1000)**2
        else:
            Dewpt = coef[0]+coef[1]*math.log(self.PP/1000)+coef[2]*math.log(self.PP/1000)**2+coef[3]*math.log(self.PP/1000)**3+coef[4]*(self.PP/1000)**0.1984

        return Dewpt
    

In [7]:
def Calculate_heat_difference(inlet, outlet, flow):
    # heat [kJ/s][kW] = average density [kg/m3] * flow [m3/s] * enthalpy difference [kj/kg]
    heat =  (inlet.density + outlet.density)/2*flow*(inlet.Enth - outlet.Enth)
    if heat>0:
        print('you are cooling down air by ', heat, 'kW')
    else:
        print('you are heating air by ', heat, 'kW')
    return heat

In [ ]:
#Below case specific scenario of calculation

In [8]:
def cool_heat(input_T, input_rel_hum, target_T, target_rel_hum):
    target_air = Air(target_T,101325,target_rel_hum)
    first_step = Air(target_air.dewpt,101325, abs_hum=target_air.abs_hum)
    input_air = Air(input_T, 101325, input_rel_hum)
    Calculate_heat_difference(input_air,first_step, 1)
    Calculate_heat_difference(first_step,target_air, 1)
    print(input_air,'\n')
    print(first_step,'\n')
    print(target_air,'\n')

In [1]:
def cool_heat_logic(input_T, input_rel_hum):
    Target_winter = Air(21,101325,35)
    Target_summer = Air(21,101325,55)
    input_air = Air(input_T,101325,input_rel_hum)
    change_humidity = False
    humified = 0
    
    print('Input air')
    print()
    print(input_air)
    
    if input_air.abs_hum < Target_winter.abs_hum:
        target_air = Air(21,101325,abs_hum=Target_winter.abs_hum)
        change_humidity = True
        print()
        print('Humification setup')
        
        print()
    elif input_air.abs_hum > Target_summer.abs_hum:
        target_air = Air(21,101325,abs_hum=Target_summer.abs_hum)
        change_humidity = True
        humified =humified+1
        print()
        print('Dehumification setup')
        print()

    else:
        target_air = Air(21,101325,abs_hum=input_air.abs_hum)
        change_humidity = False
        print()
        print('Humidity is ok')
        print()

    print('Target air')
    print()
    print(target_air)
    print()
    if change_humidity:
        first_step = Air(target_air.dewpt,101325, abs_hum=target_air.abs_hum)
        input_air = Air(input_T, 101325, input_rel_hum)
        print()
        heat1 = Calculate_heat_difference(input_air,first_step, 1)
        print()
        heat2 = Calculate_heat_difference(first_step,target_air, 1)
        humidity = target_air.abs_hum-input_air.abs_hum
        print()
        print(input_air,'\n')
        print(first_step,'\n')
        print(target_air,'\n')
    else:
        input_air = Air(input_T, 101325, input_rel_hum)
        heat1 = 0
        heat2 = Calculate_heat_difference(input_air,target_air, 1)
        humidity = 0
        print()
        print(input_air,'\n')
        print(target_air,'\n')
    print(humified)
    return heat1,heat2,humidity